In [ ]:
import pandas as pd

# Load the dataset
file_path = 'power_co_churn.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
print(data.head())
print(data.info())

In [ ]:
import pandas as pd
import glob
// write a code to read multiple excel files and merge them into one dataframe
# Get a list of all Excel files in the directory
excel_files = glob.glob('*.xlsx')

# Initialize an empty list to store the dataframes
dfs = []

# Read each Excel file and append its dataframe to the list
for file in excel_files:
    df = pd.read_excel(file)
    dfs.append(df)

# Merge all dataframes into one
merged_df = pd.concat(dfs)

# Print the merged dataframe
print(merged_df)



id: Unique identifier for the customer.
channel_sales: The sales channel through which the customer was acquired.
cons_12m: Electricity consumption of the past 12 months.
cons_gas_12m: Gas consumption of the past 12 months.
cons_last_month: Electricity consumption in the last month.
date_activ: The date the contract was activated.
date_end: The date the contract ends.
date_modif_prod: The last date the product was modified.
date_renewal: The last date the contract was renewed.
forecast_cons_12m: Forecasted electricity consumption for the next 12 months.
has_gas: Whether the customer has gas (t/f).
imp_cons: Imputed consumption.
margin_gross_pow_ele: Gross margin on power subscription.
margin_net_pow_ele: Net margin on power subscription.
nb_prod_act: Number of active products.
net_margin: Net margin.
num_years_antig: Number of years the customer has been active.
origin_up: Origin of the customer.
pow_max: Maximum power consumption.
churn: Whether the customer churned (1) or not (0).

In [ ]:
# Replace 'MISSING' with NaN for easier handling
data.replace("MISSING", pd.NA, inplace=True)

# Convert date columns to datetime, handling errors by coercing them to NaT (not a time)
date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
for col in date_cols:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Check for missing values and unique values in categorical columns to identify gibberish
print(data.isnull().sum())
print(data['channel_sales'].unique())  # Check for gibberish values

# Update: Let's handle the rows with any gibberish data, for instance in `origin_up`
print(data['origin_up'].unique())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Handling missing values: Imputing with a placeholder value for categorical columns
data['channel_sales'].fillna('unknown', inplace=True)
data['origin_up'].fillna('unknown', inplace=True)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Encode categorical variables using one-hot encoding
categorical_cols = ['channel_sales', 'origin_up', 'has_gas']
encoded_data = encoder.fit_transform(data[categorical_cols]).toarray()  # Convert to dense format using toarray()

# Create a DataFrame from the encoded data with appropriate column names
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

# Drop the original categorical columns and concatenate the encoded columns
data = data.drop(columns=categorical_cols)
data = pd.concat([data, encoded_df], axis=1)

# Splitting the dataset into training and testing sets
X = data.drop('churn', axis=1)
y = data['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the transformed DataFrame ready for modeling
X_train.head()

In [ ]:
import numpy as np

# Example transformation: calculating the number of days from a reference date
reference_date = pd.Timestamp('today')
for col in date_cols:
    X_train[col + '_days'] = (reference_date - X_train[col]).dt.days
    X_test[col + '_days'] = (reference_date - X_test[col]).dt.days

# Drop the original datetime columns from the datasets
X_train.drop(columns=date_cols, inplace=True)
X_test.drop(columns=date_cols, inplace=True)

print("Columns in X_train:", X_train.columns)
print("Columns in X_test:", X_test.columns)
print(X_train.head())

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop the ID column
#X_train = X_train.drop(columns=['id'])
#X_test = X_test.drop(columns=['id'])

# Decision Tree with Gini impurity
clf_gini = DecisionTreeClassifier(criterion='gini', random_state=42)
clf_gini.fit(X_train, y_train)
y_pred_gini = clf_gini.predict(X_test)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
print(f'Accuracy with Gini impurity: {accuracy_gini:.2f}')
print(classification_report(y_test, y_pred_gini))

# Decision Tree with Entropy
clf_entropy = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf_entropy.fit(X_train, y_train)
y_pred_entropy = clf_entropy.predict(X_test)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
print(f'Accuracy with Entropy: {accuracy_entropy:.2f}')
print(classification_report(y_test, y_pred_entropy))


In [ ]:
### CV to get the best threshold

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

# Set up the Decision Tree Classifier with entropy
tree = DecisionTreeClassifier(criterion='entropy', random_state=42)

# Define the parameter grid
param_grid = {
    'min_samples_split': range(2, 50)  # Testing splits from 2 up to 50
}

# Set up GridSearchCV to find the best parameters
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Optionally, evaluate it on the test set
best_tree = grid_search.best_estimator_
y_pred = best_tree.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.2f}".format(test_accuracy))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Predict probabilities
y_scores = best_tree.predict_proba(X_test)[:, 1]  # score = probability of positive class

# Generate ROC curve data
fpr, tpr, thresholds = roc_curve(y_test, y_scores)

# Calculate the AUC
roc_auc = auc(fpr, tpr)
print(f"The AUC is: {roc_auc:.2f}")

# Plot the ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()
